# Imports

# A

## Initialization

In [90]:
a = 0
b = 9
interval = [a,b]
Fs = [1,1]
EPSILON = 1
length = (interval[1]-interval[0])/EPSILON
while Fs[-1] < length:
    Fs.append(Fs[-1] + Fs[-2])
Fs.pop()

13

## setting Initial points

In [91]:
k = len(Fs)

x1 = a + (Fs[k-3] / Fs[k-1]) * (b - a)
x2 = a + (Fs[k-2] / Fs[k-1]) * (b - a)

ITERATIONS = k - 2

## Computing x1 and x2 in F(X)

In [92]:
for i in range(ITERATIONS):
    newInterval = []
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    if f_X1 > f_X2:
        a = x1
        x1 = x2 
        x2 = a+ (Fs[k-2]/Fs[k-1]) * (b-a)
    else:
        b = x2
        x2 = x1
        x1 = a + (Fs[k-3]/Fs[k-1]) * (b-a)
    newInterval.append(a)
    newInterval.append(b)
    print(newInterval)
    k-=1

[0, 5.625]
[0, 3.375]
[0, 2.109375]
[0.703125, 2.109375]


## Results

In [93]:
finalInterval = []
finalInterval.append(a)
finalInterval.append(b)
print(finalInterval)
minimum = (a+b) / 2
print(minimum)

[0.703125, 2.109375]
1.40625


# B

# C